In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

/root/miniconda3/envs/transformers/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 50000/50000 [00:00<00:00, 389046.64 examples/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [4]:
dataset["train"][111]

{'label': 2,
 'text': "As far as Starbucks go, this is a pretty nice one.  The baristas are friendly and while I was here, a lot of regulars must have come in, because they bantered away with almost everyone.  The bathroom was clean and well maintained and the trash wasn't overflowing in the canisters around the store.  The pastries looked fresh, but I didn't partake.  The noise level was also at a nice working level - not too loud, music just barely audible.\\n\\nI do wish there was more seating.  It is nice that this location has a counter at the end of the bar for sole workers, but it doesn't replace more tables.  I'm sure this isn't as much of a problem in the summer when there's the space outside.\\n\\nThere was a treat receipt promo going on, but the barista didn't tell me about it, which I found odd.  Usually when they have promos like that going on, they ask everyone if they want their receipt to come back later in the day to claim whatever the offer is.  Today it was one of th

In [5]:
dataset["train"][0]

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

In [7]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [8]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(dataset["train"])

,label,text
0,5 stars,"I was looking all around for dvd repair service for my xbox games, but couldn't find any shops for that around. These guys will repair your disc like new for $3. \nThanks"
1,1 star,Well I decided to give this place another chance since I really do like the food. I ended up taking a friend thinking it would be a better experience but I was sooooo wrong. I don't know what is with the women that work there but they suck at customer service. I ended up having to ask the young man that works there to take our order after three other tables that had arrived way after we arrived had recieved their food before our order was even taken. He was great as usual but it still didn't make up for the poor service we recieved that night. I will not be returning to that place ever again and I plan to speak to the owner if I can. So if you like bitchy women who neglect you and leave you starving & watching others eat then this is your place.
2,5 stars,"Love this place! Wish it was a little cheaper, but it is well worth it. The burgers have yet to let me down! The burgers are a great size, nice and crisp, cooked to perfection. Their fries are addictive and their onions rings are irresitible. This place is in such a weird location, and I really hope business picks up for them soon. Its always off putting to come in and be the only customer. Bonus points for getting root beer on tap now!! The lady who works at the register all the time is very enthusiastic and friendly, which is very refreshing."
3,5 stars,Member here for 10 years total-- I love clean! I love being able to walk away from noisy kids and go into another pool since i am a swimmer I like choices. Cool people -- members keep to themselves dont try to interfere with your workout with chit chat I LOVE THIS !! We are busy working out no talking needed ! Great classes! Great equipment and you can use other clubs too with their membership win win \nBut umm yeah don't join K? It is hard enough to find parking as it is !\n\nI went astray for a bit I will admit tried the Princeton Club to see what I was missing out on all the hoopla. What I found was old band aides that did not seem to move for days left around the pool and inches of standing water where you are supposed to wring out your suit NASTY !!\n\nI found my way back and like the Prodigal son who returns home after making mistakes I am back and I am happy!!
4,2 star,"In a word? Grim. \n\nI used to visit Biblos a lot when I was a student based on Chambers Street, but my latest visit as a fully fledged worker drone has coloured my experience differently. \n\nI came for my dinner before a visit to the museum, and boy was I disappointed. I ordered the fish and chips first of all. It arrived and the fish was in a soggy batter (urgh) and the chips were a soggy-fest too. \n\nDetermined to cheer myself up after a less than average fish and chips, I ordered a banoffee pie for dessert. Worst pudding ever. The plate arrived with a cheeky squirt of cheap skooshy cream covering the \""banoffee pie\"" (see image). By all accounts, it should have been billed as \""cheap digestive square base with tinned toffee sauce and three slices of banana on top\"". It was so bad. \n\nDo not eat here. Even though the place knocked 30% off the bill due to an offer, it was still a tenner wasted. As for drinks, couldn't comment. But the food? Like I said earlier... Grim."
5,4 stars,"Dana's Nails was the third place I went to to get my nails done in Las Vegas since I moved here 5 yrs ago. Third time's the charm! I don't get my nails done anywhere else! \n\nYes, this is an asian owned and run facility and they DO speak english. My favorite gal there is Elaine. One of my biggest things about going to any kind of spa is that the person does what I ask not what they think is \""better\"". Elaine is awesome, no questions asked, she does whatever I request and she's such a sweetheart!\n\nThe only thing that I don't like is that when they get busy, the

# 预处理数据# 
下载数据集到本地后，使用 Tokenizer 来处理文本，对于长度不等的输入数据，可以使用填充（padding）和截断（truncation）策略来处理。# 

Datasets 的 map 方法，支持一次性在整个数据集上应用预处理函# 。

下面使用填充到最大长度的策略，处理整个数据集：

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json: 100%|██████████| 49.0/49.0 [00:00<00:00, 289kB/s]
config.json: 100%|██████████| 570/570 [00:00<00:00, 3.38MB/s]
vocab.txt: 100%|██████████| 213k/213k [00:01<00:00, 173kB/s]
tokenizer.json: 100%|██████████| 436k/436k [00:01<00:00, 402kB/s]
Map: 100%|██████████| 50000/50000 [00:14<00:00, 3427.55 examples/s]


In [11]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,1 star,"This is my least favorite PT's out of all the different ones I've ever been to. If I wasn't so close to my house, I would never go. But since it's within walking distance I've been known to take the short hike here for happy hour to have a couple cheap beers, then walk home without having to add yet another over the limit driver onto the Vegas streets.\n\nThe best thing about this place is the happy hour, which is standard at all PT's...making it not that great or special. \n\nThe bad things outweigh the good unfortunately. I've always gotten horrible service here! Just horrible. And it's always so smokey in here, more so than most every other establishment I've ever been to. Those are the two main detractors for me. \n\nIf I'm just going for some grub, I'll make the mile or two drive to either the newer PT's on Buffalo and 215 or the low-key PT's at Russell and Hualapai.","[101, 1188, 1110, 1139, 1655, 5095, 22216, 112, 188, 1149, 1104, 1155, 1103, 1472, 3200, 146, 112, 1396, 1518, 1151, 1106, 119, 1409, 146, 1445, 112, 189, 1177, 1601, 1106, 1139, 1402, 117, 146, 1156, 1309, 1301, 119, 1252, 1290, 1122, 112, 188, 1439, 3179, 2462, 146, 112, 1396, 1151, 1227, 1106, 1321, 1103, 1603, 25401, 1303, 1111, 2816, 2396, 1106, 1138, 170, 2337, 10928, 23147, 117, 1173, 2647, 1313, 1443, 1515, 1106, 5194, 1870, 1330, 1166, 1103, 5310, 3445, 2135, 1103, 6554, 4324, 119, 165, 183, 165, 183, 1942, 4638, 1436, 1645, 1164, 1142, 1282, 1110, 1103, 2816, 2396, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


# 数据抽样# 
使用 1000 个数据样本，在 BERT 上演示小规模训练（基于 Pytorch Trainer# 

shuffle()函数会随机重新排列列的值。如果您希望对用于洗牌数据集的算法有更多控制，可以在此函数中指定generator参数来使用不同的numpy.random.Generator。

In [12]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

# 微调训练配置# 
加载 BERT 模型# 
警告通知我们正在丢弃一些权重（vocab_transform 和 vocab_layer_norm 层），并随机初始化其他一些权重（pre_classifier 和 classifier 层）。在微调模型情况下是绝对正常的，因为我们正在删除用于预训练模型的掩码语言建模任务的头部，并用一个新的头部替换它，对于这个新头部，我们没有预训练的权重，所以库会警告我们在用它进行推理之前应该对这个模型进行微调，而这正是我们要做的事情。

In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

model.safetensors: 100%|██████████| 436M/436M [05:27<00:00, 1.33MB/s] 
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initi

# 训练超参数（TrainingArguments）# 
完整配置参数与默认值：https://huggingface.co/docs/transformers/v4.36.1/en/main_classes/trainer#transformers.TrainingArguments# 

源代码定义：https://github.com/huggingface/transformers/blob/v4.36.1/src/transformers/training_args.py#L16# 1

最重要配置：模型权重保存路径(output_dir)

In [14]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased-finetune-yelp"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=model_dir,                # 权重保存路径
                                  per_device_train_batch_size=16,      # 训练批次：每个GPU同时处理16个样本
                                  num_train_epochs=5,                  # 训练轮数：模型将经历5个训练周期
                                  logging_steps=100)                   # 记录日志的步数间隔：每100个训练步骤将会记录一次日志。日志通常包含训练过程中的重要信息，如损失值、准确性等，以及模型检查点的保存

2024-02-22 13:52:44.343660: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-22 13:52:44.949336: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 13:52:44.949419: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 13:52:45.022485: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 13:52:45.244821: I tensorflow/core/platform/cpu_feature_guar

In [15]:
# 完整的超参数配置
print(training_args)

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.NO,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=HubStrategy.EVERY_SAVE,

# 训练过程中的指标评估（Evaluate)# 
Hugging Face Evaluate 库 支持使用一行代码，获得数十种不同领域（自然语言处理、计算机视觉、强化学习等）的评估方法。 当前支持 完整评估指标：https://huggingface.co/evaluate-metric# 

训练器（Trainer）在训练过程中不会自动评估模型性能。因此，我们需要向训练器传递一个函数来计算和报告指标# 。

Evaluate库提供了一个简单的准确率函数，您可以使用evaluate.load函数加载

In [18]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

# 接着，调用 compute 函数来计算预测的准确率。

# 在将预测传递给 compute 函数之前，我们需要将 logits 转换为预测值（所有Transformers 模型都返回 logits）。

In [19]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 训练过程指标监控# 
通常，为了监控训练过程中的评估指标变化，我们可以在TrainingArguments指定evaluation_strategy参数，以便在 epoch 结束时报告评估指标。

In [21]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

# 开始训练# 
实例化训练器（Trainer）# 
kernel version 版本问题：暂不影响本示例代码运行

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [24]:
trainer.train()

/root/miniconda3/envs/transformers/lib/python3.11/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.359700,1.155565,0.502000
2,0.972000,0.972141,0.582000
3,0.639300,0.970710,0.596000


TrainOutput(global_step=189, training_loss=1.0430158837131722, metrics={'train_runtime': 344.8174, 'train_samples_per_second': 8.7, 'train_steps_per_second': 0.548, 'total_flos': 789354427392000.0, 'train_loss': 1.0430158837131722, 'epoch': 3.0})

In [25]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))

In [26]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 0.9985702633857727,
 'eval_accuracy': 0.63,
 'eval_runtime': 2.9967,
 'eval_samples_per_second': 33.37,
 'eval_steps_per_second': 4.338,
 'epoch': 3.0}

# 保存模型和训练状态# 
使用 trainer.save_model 方法保存模型，后续可以通过 from_pretrained() 方法重新加载# 
使用 trainer.save_state 方法保存训练状态

In [27]:
trainer.save_model(model_dir)

In [28]:
trainer.save_state()

# Homework: 使用完整的 YelpReviewFull 数据集训练，看 Acc 最高能到多少

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=model_dir,
                                  evaluation_strategy="epoch", 
                                  per_device_train_batch_size=16,
                                  num_train_epochs=3,
                                  logging_steps=30)

In [30]:
full_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(650000))
full_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(50000))

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset, 
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)

In [32]:
trainer.train()

/root/miniconda3/envs/transformers/lib/python3.11/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

#  每次调入16个样本进行训练
#  训练完16个样本为一次训练步骤，模型会进行一次前向传播和反向传播
#  650000个样本，所需训练步骤为650000/16=40625步
#  训练轮次epoch为3次，因此，全部训练步骤为40625*3=121875步
#  每30步形成一次训练日志记录，因此记录条数为121875/30=4062.5，即4063次记录